In [6]:
import numpy as np 
import matplotlib.pyplot as plt 
import src_experiment as scf
import geobin as gb 
import pathlib as pl
import torch
from sklearn.datasets import make_moons


training_seed = 42
testing_seed = 41
inference_seed = 40



# Read config file and/or parse cmb-line arguments
# args = scf.get_args()



# # Check if the experiment path exists, if not, create 
# experiment_path = scf.get_path_to_experiment_storage(args.experiment_name)
# state_dict_path = experiment_path/"state_dicts"
# if not state_dict_path.exists():
#     scf.createfolders(state_dict_path)

# Make the data
train_data = scf.make_moon_dataloader(n_samples=10000, noise=0.15, random_state=training_seed, batch_size=250)
test_data = scf.make_moon_dataloader(n_samples=2000, noise=0.15, random_state=testing_seed, batch_size=250)

# # Set up the model based on the parsed arguments or the config file
# model = scf.NeuralNet(input_size=2, hidden_sizes=args.architecture, num_classes=1)

# # Train the model on the training data
# scf.train_model(model, train_data, test_data, args.epochs, state_dict_path, args.experiment_name, SAVE_STATES=False, save_everyth_epoch=10)


In [15]:
for i, (x,y) in enumerate(train_data):
    for x in x:
        print(x)

tensor([ 0.5363, -0.0842], dtype=torch.float64)
tensor([-0.5661, -0.0364], dtype=torch.float64)
tensor([0.3885, 0.7442], dtype=torch.float64)
tensor([-0.8026,  0.1277], dtype=torch.float64)
tensor([-0.9268,  0.3910], dtype=torch.float64)
tensor([ 0.1159, -0.3393], dtype=torch.float64)
tensor([1.0848, 0.2719], dtype=torch.float64)
tensor([-0.2490,  0.2381], dtype=torch.float64)
tensor([ 0.3020, -0.2306], dtype=torch.float64)
tensor([0.9516, 0.1803], dtype=torch.float64)
tensor([-0.0159,  0.9683], dtype=torch.float64)
tensor([ 1.0236, -0.5841], dtype=torch.float64)
tensor([0.2409, 0.2711], dtype=torch.float64)
tensor([0.0942, 0.2563], dtype=torch.float64)
tensor([-0.7669,  0.6262], dtype=torch.float64)
tensor([-0.0926, -0.0024], dtype=torch.float64)
tensor([0.9422, 0.4389], dtype=torch.float64)
tensor([2.0265, 0.3728], dtype=torch.float64)
tensor([ 0.8560, -0.4308], dtype=torch.float64)
tensor([ 1.2087, -0.2193], dtype=torch.float64)
tensor([1.7833, 0.5705], dtype=torch.float64)
tensor([